##### Improving Prediction Accuracy of Sepsis Mortality using Natural Language Processing
## Tyler Kelly

# Machine Learning Pipeline

## Set Up and Install Dependencies

In [ ]:
#pip install datasets transformers pandas shap

## Part 0 Preprocessing (Pull Code from Author's ipynb)

The following code is adapted from the github repository 'https://github.com/yuyinglu2000/Sepsis-Mortality'

In [1]:
#AC

import warnings
warnings.filterwarnings("ignore")
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import lightgbm as lgb

### Data Mapping

Begin by creating bigquery search to get the 38 unique features

In [ ]:
#df = pd.read_csv('Data/data_raw.csv')

In [ ]:
df_raw.shape
# Expect to get a dataframe 808188x38 (38 old columns)

In [ ]:
#AC

# regroup the race
race_mapping = {
    'WHITE': 'White',
    'HISPANIC OR LATINO': 'Hispanic or Latin',
    'BLACK/AFRICAN AMERICAN': 'Black or African American',
    'BLACK/CARIBBEAN ISLAND': 'Black or African American',
    'HISPANIC/LATINO - DOMINICAN': 'Hispanic or Latin',
    'HISPANIC/LATINO - CENTRAL AMERICAN': 'Hispanic or Latin',
    'HISPANIC/LATINO - GUATEMALAN': 'Hispanic or Latin',
    'HISPANIC/LATINO - PUERTO RICAN': 'Hispanic or Latin',
    'HISPANIC/LATINO - SALVADORAN': 'Hispanic or Latin',
    'HISPANIC/LATINO - HONDURAN': 'Hispanic or Latin',
    'HISPANIC/LATINO - MEXICAN': 'Hispanic or Latin',
    'HISPANIC/LATINO - CUBAN': 'Hispanic or Latin',
    'HISPANIC/LATINO - COLUMBIAN': 'Hispanic or Latin',
    'BLACK/CAPE VERDEAN': 'Black or African American',
    'BLACK/AFRICAN': 'Black or African American',
    'SOUTH AMERICAN': 'Hispanic or Latin',
    'WHITE - BRAZILIAN': 'Hispanic or Latin',
    'WHITE - OTHER EUROPEAN': 'White',
    'WHITE - RUSSIAN': 'White',
    'WHITE - EASTERN EUROPEAN': 'White',
    'ASIAN': 'Others race',
    'ASIAN - SOUTH EAST ASIAN': 'Others race',
    'ASIAN - CHINESE': 'Others race',
    'ASIAN - ASIAN INDIAN': 'Others race',
    'ASIAN - KOREAN': 'Others race',
    'AMERICAN INDIAN/ALASKA NATIVE': 'Others race',
    'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 'Others race',
    'MULTIPLE RACE/ETHNICITY': 'Others race',
    'PORTUGUESE': 'Others race',
    'UNKNOWN': 'Others race',
    'OTHER': 'Others race',
    'UNABLE TO OBTAIN': 'Others race',
    'PATIENT DECLINED TO ANSWER': 'Others race'
}

df_raw['race'] = df_raw['race'].map(race_mapping)

In [ ]:
#AC

import pandas as pd

# Assuming df is your DataFrame
# df = ... (your DataFrame)

# Define a mapping for antibiotics to their respective groups
antibiotic_mapping = {
    'Gentamicin Sulfate': 'Aminoglycoside',
    'Tobramycin Sulfate': 'Aminoglycoside',
    'Streptomycin Sulfate': 'Aminoglycoside',
    'Neomycin Sulfate': 'Aminoglycoside',
    'Neomycin/Polymyxin B Sulfate': 'Aminoglycoside',
    'Meropenem': 'Carbapenem',
    'Meropenem Graded Challenge': 'Carbapenem',
    'Vancomycin': 'Glycopeptide',
    'Vancomycin Oral Liquid': 'Glycopeptide',
    'Vancomycin Antibiotic Lock': 'Glycopeptide',
    'Vancomycin Enema': 'Glycopeptide',
    'Vancomycin Intrathecal': 'Glycopeptide',
    'Vancomycin Ora': 'Glycopeptide',
    'Linezolid': 'Oxazolidinone',
    'Linezolid Suspension': 'Oxazolidinone',
    'Penicillin G Benzathine': 'Penicillin',
    'Penicillin G Potassium': 'Penicillin',
    'Penicillin V Potassium': 'Penicillin',
    'Sulfameth/Trimethoprim': 'Sulfonamide',
    'Sulfameth/Trimethoprim DS': 'Sulfonamide',
    'Sulfameth/Trimethoprim SS': 'Sulfonamide',
    'Sulfamethoxazole-Trimethoprim': 'Sulfonamide',
    'Sulfameth/Trimethoprim Suspension': 'Sulfonamide',
    'Tetracycline': 'Tetracycline',
    'Tetracycline HCl': 'Tetracycline'
    # Add more mappings as needed
}

# Applying the mapping to the 'antibiotic' column
df_raw['antibiotic'] = df_raw['antibiotic'].map(antibiotic_mapping)

In [ ]:
#AC
df_raw['antibiotic'].unique()

### Get Dummy Variables

In [ ]:
#AC
df_encoded = pd.get_dummies(df_raw, columns=df_raw.select_dtypes(include=['object']).columns)
df_dropped = df_encoded.dropna()
df_dropped.info()

In [ ]:
### Check empty values *for tetracycline* ###
#AC
empty_values = df_dropped['antibiotic_Tetracycline'].isnull().any()
empty_values

In [ ]:
df.info

After applying get_dummy_variables there is now 53 columns

### Drop Duplications

In [ ]:
#AC with minor edits

duplicated_rows_mask = df_dropped['subject_id'].duplicated(keep=False)

# Extract the duplicated rows
duplicated_rows = df_dropped[duplicated_rows_mask]
new_data  = df_dropped.drop_duplicates()
duplicated_rows_mask = new_data['subject_id'].duplicated(keep=False)

# Extract the duplicated rows
duplicated_rows = new_data[duplicated_rows_mask]
# Separate out columns based on data types
int_float_cols = new_data.select_dtypes(include=['int64', 'float64']).columns
uint8_cols = new_data.select_dtypes(include=['uint8']).columns

# Sort dataframe
# For int and float columns: sort in descending order so that larger values come first
df_raw = new_data.sort_values(by=list(int_float_cols), ascending=False)

# For uint8 columns: sort in descending order so that 1 comes before 0
df_raw = df_raw.sort_values(by=list(uint8_cols), ascending=False)

# Drop duplicates based on subject_id, keeping the first (which are the desired rows after sorting)
df_reduced = df_raw.drop_duplicates(subset='subject_id', keep='first')

# Reset index if needed
df_reduced = df_reduced.reset_index(drop=True)
pd.set_option('display.max_columns', None)
df_reduced

In [ ]:
df_reduced.shape

After reducing the dataframe we get the 6401 patients reported in the paper.

In [ ]:
df_reduced.info

In [ ]:
df_subject = df_reduced['subject_id']

In [ ]:
df_subject.to_csv("data_subject_id_ready_to_query.csv", index=False)

## Part 1 Upload data_ready_to_merge.csv to BigQuery

Upload the subject_id dataset to bigquery and merge the dataset to radiology and discharge notes separately. Save the downloaded file to downloads (or find a way to save it directly to my BIOST 2021 Thesis / Main ) as
'data_radiology_notes.csv' and 'data_discharge_notes.csv' then joing below.

Place SQL code chunks below (if time write them into the script)

# READ THIS

data_full_notes_old.csv uses an old outdated dataset but I haven't figured out the correct sql to get the correct dataset at 6401.

For now, I will use `df_old` when using the outdated dataset and `df` when I get the new corrected one.

To get the radiology and discharge notes, I uploaded the data_after_cleaning table to big query and used the below code sql query to get a table with
data_after_cleaning joined to the notes columns found in the mimic-iv discharge and radiology tables.

Above, using the correct subject_id list to get the 6401 patients, I use the same method to obtain the rad_notes and discharge_notes tables using bigquery, and join in an identical manner.

In [2]:
df = pd.read_csv('Data/Old/data_full_notes_old.csv') # df_old
# change after fixing sql

In [3]:
df.shape

(303994, 58)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303994 entries, 0 to 303993
Data columns (total 58 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   int64_field_0                          303994 non-null  int64  
 1   subject_id                             303994 non-null  int64  
 2   hospital_expire_flag                   303994 non-null  int64  
 3   max_age                                303994 non-null  int64  
 4   los_icu                                303994 non-null  float64
 5   first_hosp_stay                        303994 non-null  bool   
 6   suspected_infection                    303994 non-null  int64  
 7   sofa_score                             303994 non-null  int64  
 8   sepsis3                                303994 non-null  bool   
 9   avg_urineoutput                        303994 non-null  float64
 10  glucose_min                            303994 non-null  

## Part 2 Truncate Notes for Word2Vec

In [5]:
from joblib import Parallel, delayed
import pandas as pd
import re
import multiprocessing

In [6]:
# === Step 1: Define cleaning function - Clean individual note text ===
def clean_text(text):
    if pd.isna(text):
        return ""
    text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
    text = re.sub(r'_+', '', text)    # Remove underlines
    text = re.sub(r'[^\w\s.,:;!?()\-\n]', '', text)  # Remove junk, keep clinical symbols
    return text.strip()

In [7]:
# === Step 2: Function to process one group. Process a group into (subject_id, note_type, combined_notes) ===
def process_group(record):
    subject_id = record['subject_id']
    note_type_1 = record['note_type_1'] #this column identifies addendums and base notes to just be 'radiology' notes
    texts = record['text']
    cleaned_notes = [clean_text(text) for text in texts]
    combined_notes = " ".join(cleaned_notes)
    return {
        'subject_id': subject_id,
        'note_type_1': note_type_1,
        'combined_notes': combined_notes
    }

In [8]:
# === Step 3: Group the notes. Load and group your data ===
# Replace this with your actual loading logic / dataframe
# Data loaded above as `df`
grouped_df = (
    df.groupby(['subject_id', 'note_type_1'])['text']
    .apply(list)
    .reset_index()
)

records = grouped_df.to_dict('records')

In [9]:
# === Step 4: Parallel processing with joblib ===

#this num_cores variable is used later for other parallel processing jobs
num_cores = multiprocessing.cpu_count() - 1

processed = Parallel(n_jobs=num_cores)(
    delayed(process_group)(record) for record in records
)

The following step creates a long dataframe for notes per patient by type (some patients have radiology, discharge, both, or none for notes), it is later converted to wide to potentially compare NLP to see if discharge notes significantly provide insight into mortality rates.

In [10]:
# === Step 5: Create DataFrame and save to csv ===
nlp_long_df = pd.DataFrame(processed).sort_values(by=['subject_id', 'note_type_1'])

nlp_long_df.to_csv("Data/Old/data_trunc_notes_old.csv", index=False) # change after fixing sql

In [54]:
nlp_long_df.shape

(10402, 3)

In [11]:
# === Step 6: Pivot to wide format for multiple columns ===
# Convert note_type to columns like 'Radiology_notes', etc.
nlp_wide_df = nlp_long_df.pivot(
    index='subject_id',
    columns='note_type_1',
    values='combined_notes'
).reset_index()

nlp_wide_df.columns.name = None # Remove category label

# Rename columns to make clear
nlp_wide_df = nlp_wide_df.rename(columns={
    'radiology': 'Radiology_notes',
    'discharge': 'Discharge_summary_notes'
})

nlp_wide_df = nlp_wide_df.fillna("") #fills NA columns with empty strings

# Save
nlp_wide_df.to_csv("Data/Old/data_trunc_notes_wide_old.csv", index=False) # change after fixing sql

In [12]:
# === Step 7: Combine Radiology and Discharge notes per subject_id ===
nlp_combined_df = nlp_wide_df.copy()

# Concatenate the two columns into one
nlp_combined_df['combined_notes'] = (
    nlp_combined_df['Radiology_notes'].str.strip() + " " +
    nlp_combined_df['Discharge_summary_notes'].str.strip()
).str.strip()

# Combined DataFrame with just subject_id + combined text
nlp_combined_notes_df = nlp_combined_df[['subject_id', 'combined_notes']]

# Save
nlp_combined_notes_df.to_csv("Data/Old/data_trunc_notes_combined_old.csv", index=False) # change after fixing sql

The code chunk below creates `nlp_ready_df`, a dataframe that includes a row for each subject_id and appends 3 new columns with all text truncated based on radiology notes, discharge notes, and combined radiology and discharged notes. I lose information related to the note_id, note_id_type, and more importantly charttime, but here it allows word2vec to work properly. This part of my thesis focuses more on using embeddings from the clinical text to aid in predicting mortality rather than finding the best way to do it (i.e. finding the best time of day where it is more likely to happen than not, or finding the best drug at predicting it, etc.).

In [52]:
# === Step 8: Join nlp_wide_df and nlp_combined_notes_df to data_after_cleaning ===

# === Step i: Load the original cleaned dataset, df_reduced ===
# ie this is df_reduced

# For now, this is `df_clean` the cleaned dataset.

df_clean = pd.read_csv('Data/Old/data_after_cleaning.csv')

# === Step ii: Merge df_reduced with nlp_wide_df ===
# This adds the radiology and discharge notes as 2 new columns to df_reduced
# use data_clean until df_reduced is finalized

nlp_ready_df = df_clean.merge(
    nlp_wide_df,
    on='subject_id',
    how='left'
)

# === Step iii: Merge with combined notes ===
# This adds one new column of all notes combined together as a single note (per patient) to the nlp_ready_df above
nlp_ready_df = nlp_ready_df.merge(
    nlp_combined_notes_df,
    on='subject_id',
    how='left'
)



# Save
nlp_ready_df.to_csv("Data/Old/data_nlp_ready_old.csv", index=False)


In [24]:
# === Step 9: Check shape of dataframes ===
nlp_wide_df.shape

(5208, 3)

In [25]:
nlp_combined_notes_df.shape

(5208, 2)

In [26]:
df_reduced.shape

NameError: name 'df_reduced' is not defined

In [27]:
nlp_ready_df.shape

(5208, 51)

In [28]:
nlp_ready_df.columns

Index(['subject_id', 'hospital_expire_flag', 'max_age', 'los_icu',
       'first_hosp_stay', 'suspected_infection', 'sofa_score', 'sepsis3',
       'avg_urineoutput', 'glucose_min', 'glucose_max', 'glucose_average',
       'sodium_max', 'sodium_min', 'sodium_average', 'diabetes_without_cc',
       'diabetes_with_cc', 'severe_liver_disease', 'aids', 'renal_disease',
       'heart_rate_min', 'heart_rate_max', 'heart_rate_mean', 'sbp_min',
       'sbp_max', 'sbp_mean', 'dbp_min', 'dbp_max', 'dbp_mean',
       'resp_rate_min', 'resp_rate_max', 'resp_rate_mean', 'spo2_min',
       'spo2_max', 'spo2_mean', 'coma', 'albumin',
       'race_Black or African American', 'race_Hispanic or Latin',
       'race_Others race', 'race_White', 'antibiotic_Vancomycin',
       'antibiotic_Vancomycin Antibiotic Lock', 'antibiotic_Vancomycin Enema',
       'antibiotic_Vancomycin Intrathecal',
       'antibiotic_Vancomycin Oral Liquid', 'gender_F', 'gender_M',
       'Discharge_summary_notes', 'Radiology_note

In [29]:
nlp_ready_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5208 entries, 0 to 5207
Data columns (total 51 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   subject_id                             5208 non-null   int64  
 1   hospital_expire_flag                   5208 non-null   int64  
 2   max_age                                5208 non-null   int64  
 3   los_icu                                5208 non-null   float64
 4   first_hosp_stay                        5208 non-null   bool   
 5   suspected_infection                    5208 non-null   int64  
 6   sofa_score                             5208 non-null   int64  
 7   sepsis3                                5208 non-null   bool   
 8   avg_urineoutput                        5208 non-null   float64
 9   glucose_min                            5208 non-null   int64  
 10  glucose_max                            5208 non-null   int64  
 11  gluc

## Part 3 Create Note File for Word2Vec

In [30]:
import pandas as pd

# Write the 'Radiology_notes' column to a text file, one line per document
with open("Data/Old/W2V_old/w2v_Radiology_notes.txt", "w", encoding="utf-8") as f: # change after fixing sql
    for line in nlp_ready_df["Radiology_notes"]:
        f.write(str(line).strip() + "\n")

In [32]:
# Write the 'Discharge_summary_notes' column to a text file, one line per document
with open("Data/Old/W2V_old/w2v_Discharge_notes.txt", "w", encoding="utf-8") as f: # change after fixing sql
    for line in nlp_ready_df["Discharge_summary_notes"]:
        f.write(str(line).strip() + "\n")

In [33]:
# Write the 'combined_notes' column to a text file, one line per document
with open("Data/Old/W2V_old/w2v_combined_notes.txt", "w", encoding="utf-8") as f: # change after fixing sql
    for line in nlp_ready_df["combined_notes"]:
        f.write(str(line).strip() + "\n")

## Part 4 Prepare Word2Vec - Proceed to main.rmd

## Part 5 Model Training - Proceed to Sepsis_Model_Training.ipynb

After completing / running / saving models in model training, upload them into the workspace in the following code chunks if necessary.

## Part 6 Create Dataset for Bert

Metadata csv files for radiology and discharge are created and saved earlier

In [34]:
import os
import pandas as pd
from joblib import Parallel, delayed
import multiprocessing

import os
import pandas as pd
from joblib import Parallel, delayed

def write_single_note_clean(row, output_dir):
    try:
        subject_id = str(row["subject_id"]).strip()
        note_id = str(row["note_id"]).strip()
        note_text = row.get("text", "")

        # Ensure note_text is a clean string
        if not isinstance(note_text, str):
            note_text = "" if pd.isna(note_text) else str(note_text)
        note_text = note_text.strip()

        # Skip empty notes
        if not note_text:
            return

        # Make subject directory
        subject_dir = os.path.join(output_dir, subject_id)
        os.makedirs(subject_dir, exist_ok=True)

        # Save note
        file_path = os.path.join(subject_dir, f"{subject_id}_{note_id}.txt")
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(note_text)

    except Exception as e:
        print(f"Error writing note {row.get('note_id', 'unknown')}: {e}")



# Change output directories / metadata for discharge notes
def write_mimic_notes_parallel_for_bert(df, 
                                        output_dir="BERT/BERT_old/rad_notes", 
                                        metadata_csv="BERT/BERT_old/metadata_rad_notes_old.csv", 
                                        n_jobs=num_cores):
    os.makedirs(output_dir, exist_ok=True)
    rows = df.to_dict("records")

    # Write notes in parallel
    Parallel(n_jobs=n_jobs, prefer="threads")(
        delayed(write_single_note_clean)(row, output_dir) for row in rows
    )

    # Save metadata
    metadata_cols = ['subject_id', 'note_id', 'note_type_1', 'charttime']
    metadata_df = df.dropna(subset=['subject_id', 'note_id'])
    metadata_df = metadata_df[metadata_cols]
    metadata_df.to_csv(metadata_csv, index=False)

    print(f"✅ Notes saved to: {output_dir}")
    print(f"✅ Metadata saved to: {metadata_csv}")


In [35]:
from datasets import Dataset
from joblib import Parallel, delayed

def create_bert_dataset_from_notes(metadata_csv, notes_root_dir):
    df = pd.read_csv(metadata_csv)

    def load_text(row):
        subject_id = str(row['subject_id'])
        note_id = str(row['note_id'])
        file_path = os.path.join(notes_root_dir, subject_id, f"{subject_id}_{note_id}.txt")
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read().strip()
        except FileNotFoundError:
            return ""

    texts = Parallel(n_jobs=num_cores)(delayed(load_text)(row) for _, row in df.iterrows())
    df['text'] = texts
    df = df[df['text'].str.strip() != ""]  # Remove blanks

    dataset = Dataset.from_pandas(df.reset_index(drop=True))
    print(f"✅ Dataset created with {len(dataset)} entries")
    return dataset

In [36]:
from transformers import AutoTokenizer

def tokenize_bert_dataset(dataset, model_name='emilyalsentzer/Bio_ClinicalBERT', text_column='text'):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def tokenize_function(example):
        return tokenizer(
            example[text_column],
            truncation=True,
            padding='max_length',
            max_length=512
        )

    print("🔄 Tokenizing dataset...")
    tokenized = dataset.map(tokenize_function, batched=True)
    return tokenized


In [37]:
def save_tokenized_dataset(tokenized_dataset, output_path="BERT/BERT_old/tokenized_dataset"):
    tokenized_dataset.save_to_disk(output_path)
    print(f"✅ Tokenized dataset saved to: {output_path}")


In [38]:
# Load Notes
bert_rad = pd.read_csv("Data/Old/data_full_notes_old.csv")

In [39]:
# Test mimic_notes_parallel_for_bert
import time

sample_df = df.sample(100)
start = time.time()
write_mimic_notes_parallel_for_bert(df = sample_df, output_dir="BERT/BERT_old/rad_notes_test", metadata_csv="BERT/BERT_old/metadata_rad_notes_old_test.csv", n_jobs=1)
print(f"Time for 100 notes: {time.time() - start:.2f} seconds")

✅ Notes saved to: BERT/BERT_old/rad_notes_test
✅ Metadata saved to: BERT/BERT_old/metadata_rad_notes_old_test.csv
Time for 100 notes: 0.11 seconds


In [40]:
# Extract and save notes
write_mimic_notes_parallel_for_bert(bert_rad)

✅ Notes saved to: BERT/BERT_old/rad_notes
✅ Metadata saved to: BERT/BERT_old/metadata_rad_notes_old.csv


In [42]:
# Test Rebuild dataset
import time

start = time.time()
create_bert_dataset_from_notes("BERT/BERT_old/metadata_rad_notes_old_test.csv", "BERT/BERT_old/rad_notes_test")
print(f"Time for 100 notes: {time.time() - start:.2f} seconds")

✅ Dataset created with 100 entries
Time for 100 notes: 0.10 seconds


In [43]:
# Rebuild Dataset
rad_bert_dataset_rebuilt = create_bert_dataset_from_notes("BERT/BERT_old/metadata_rad_notes_old.csv", "BERT/BERT_old/rad_notes")

✅ Dataset created with 303994 entries


In [44]:
# Tokenize with Clinical BERT
tokenized_dataset = tokenize_bert_dataset(rad_bert_dataset_rebuilt)

🔄 Tokenizing dataset...


Map:   0%|          | 0/303994 [00:00<?, ? examples/s]

In [45]:
# Save for later
save_tokenized_dataset(tokenized_dataset)

Saving the dataset (0/4 shards):   0%|          | 0/303994 [00:00<?, ? examples/s]

✅ Tokenized dataset saved to: BERT/BERT_old/tokenized_dataset


## Part 7 Clinical BERT ML Pipeline - Proceed to clinical_bert_ml.ipynb